## Load

In [10]:
from optimize import *
from full_model import *


Starting MATLAB engine...
-------------------------
Elapsed: 22.70 sec


In [11]:
data_loaded = loadmat("./saved/rescaled/Coil20")
images_norm = data_loaded['X']
images_norm_val = data_loaded['X_val']
labels = data_loaded['Y'].reshape(-1)
labels_val = data_loaded['Y_val'].reshape(-1)

In [12]:
load_path = "./saved/models/coil20/256.64_10000.10.3_0.2098"
hidden_dims = [256,64,256]

In [58]:
result1 = load("./optims/train/1_SSC/256.64_10000.10.3.opt")
result2 = load("./optims/train/2_SSC+AE/256.64_10000.10.3.opt")
result3 = load("./optims/train/3_Global/256.64_10000.10.3.opt")
result4 = load("./optims/train/4_Glob+Symm/256.64_10000.10.3.opt")
result5 = load("./optims/train/5_Conjoin/256.64_10000.10.3.opt")
result6 = load("./optims/train/6_Conj+Init/256.64_10000.10.3.opt")

In [59]:
results = [result1, result2, result3, result4, result5, result6]

## Re-eval

In [60]:
optimums = [res_optimum(result)[0] for result in results]

In [61]:
models = [run_ssc, run_ae, run_model, run_model, run_model, run_model]

In [62]:
spaces = [['alpha'],
          ['lr', 'batch_num', 'lambda2', 'alpha2'],
          ['lr', 'batch_num', 'alpha1', 'lambda1', 'lambda2', 'alpha2'],
          ['lr', 'batch_num', 'alpha1', 'lambda1', 'lambda2', 'alpha2'],
          ['lr', 'batch_num', 'lambda1', 'lambda2', 'lambda3'],
          ['lr', 'batch_num', 'alpha1', 'lambda1', 'lambda2', 'lambda3']]

In [63]:
params = [{'images_norm':images_norm, 'labels':labels},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims, 'trainC':False, 'symmC':False},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims, 'trainC':False, 'symmC':True},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims, 'trainC':True, 'giveC':False},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims, 'trainC':True, 'giveC':False}]

In [ ]:
from copy import copy

def run(idx, hypers=optimums[idx], seed=0, verbose=True):
    par = copy(params[idx])
    par.update(zip(spaces[idx], hypers))
    return models[idx](seed=seed, verbose=verbose, **par)

In [73]:
def run_mult(idx, hypers=optimums[idx], seeds=range(100, 105), verbose=False):
    log_sum = 0
    for seed in seeds:
        log_sum += np.log(run(idx, hypers, seed, verbose))
    return np.exp(log_sum / len(seeds))

In [56]:
for idx in range(6):
    print(idx, run_mult(idx))

0 [0.39099743 0.2419614  0.53952853]
1 [0.41375444 0.24220494 0.56159482]
2 [0.38009483 0.26630685 0.53243602]
3 [0.45840264 0.27189531 0.64254757]


C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


4 [0.52811586 0.43392175 0.68967471]


C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


5 [0.5283546  0.40890772 0.67817871]


IndexError: list index out of range

In [ ]:
k = len(np.unique(labels))
grps = eng.SSC_modified(k, 0, False, 6.0, False, 1, 1e-20, 100, True, -1, False, **mlab_kwargs)
labels_pred = np.asarray(grps, dtype=np.int32).flatten()

## Sensitivity analysis

In [ ]:
from copy import copy

mults = [10**-1, 10**-0.5, 10**0.5, 10**1]
for idx in range(len(results)):
    # for each model
    xs = []
    ys = []
    hypers = copy(optimums[idx])
    for i in range(len(spaces[idx])):
        # for each hyperparameter
        x = []
        y = []
        hyper = hypers[i]
        for mult in mults:
            # for each multiplied value of said hyperparameter
            hypers[i] = hyper * mult
            x.append(hyper * mult)
            y.append(run_mult(idx, hypers))
        hypers[i] = hyper
        xs.append(x)
        ys.append(y)
    print(idx, x, y)

[0.607566580310015]
[0.607566580310015]
[0.607566580310015]
[0.607566580310015]
[0.607566580310015]
[1.9212942239792576]
[1.9212942239792576]
[1.9212942239792576]
[1.9212942239792576]
[1.9212942239792576]
